https://arxiv.org/abs/1706.03762 (original paper) <br/>
https://jalammar.github.io/illustrated-transformer/ (blog illustrated explanation) <br/>
https://nlp.seas.harvard.edu/2018/04/03/attention.html (paper with annotation with code) <br/>
https://www.youtube.com/watch?v=kCc8FmEb1nY (karpathy gpt from scratch video) <br/>
https://github.com/karpathy/ng-video-lecture (karpathy gpt from scratch notebooks) <br/>

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [18]:
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
n_embd = 384 # embedding dim
n_head = 6 # dim of key, query, value, embedding
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
dropout = 0.2

In [4]:
torch.manual_seed(1337);

In [5]:
with open('./data/tinyshakespeare.txt', 'r') as f:
    text = f.read()

In [10]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l : "".join(itos[c] for c in l)

In [11]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [13]:
def get_batch(split):
    data = train_data if split=='train' else val_data
    idx = torch.randint(len(data)-block_size, (batch_size,))
    xs = torch.stack([data[i:i+block_size] for i in idx])
    ys = torch.stack([data[i+1:i+block_size+1] for i in idx])
    xs, ys = xs.to(device), ys.to(device)
    return xs, ys

In [16]:
@torch.no_grad()
def estimate_loss(model):
    o = {}
    model.eval()
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        o[split] = losses.mean().item()
    model.train()
    return o

In [57]:
# self attention single head.
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, n_head, bias=False)
        self.query = nn.Linear(n_embd, n_head, bias=False)
        self.value = nn.Linear(n_embd, n_head, bias=False)
        
        self.dropout = nn.Dropout(dropout)
        
        # .register_buffer: to store non-trainable tensors.
        # torch.tril will create lower triangle matrix.
        # used for masking future chars. 
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    
    def forward(self, x):
        # x shape: B,T,C
        B,T,C = x.shape
        
        # get the key embedding of each token in the context.
        k = self.key(x) # shape: B,T,h
        
        # get the query embedding of each token in the context.
        q = self.query(x) # shape: B,T,h
        
        # calc dot product each query embedding with key embedding, this will give
        # sort of 'affinitity or attention' score for each token against each other token.
        # k.shape[-1]**0.5 : this is kind of initialization, graident stabilzing factor, k.shape[-1] is the
        # head size (n_head), this suggested scaling factor as per paper.
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # shape: B,T,h -> B,T,T 
        
        # masking of future token. e.g first can only see itself, and second one see itself and prev
        # one.
        # float('-inf'): because during exponential those will go to zero.
        # todo: the need for tril[:T, :T] not clear yet because tril already defined in T shae???
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # shape: B,T,T
        
        # convert wei is probs
        wei = F.softmax(wei, dim=-1) # shape: B,T,T
        wei = self.dropout(wei)
        

In [50]:
x = torch.ones(5, 3, 3)
x.shape

torch.Size([5, 3, 3])

In [51]:
tr = torch.tril(torch.ones(3, 3))

In [53]:
tr[:3, :3].shape

torch.Size([3, 3, 3])

In [54]:
tr.shape

torch.Size([3, 3])

In [55]:
tr

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [56]:
tr[:3, :3]

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])